UPLOAD NYAYANUMANA_KNOWLEDGE.CSV


GET TOP X DATAPOINTS FROM ORIGINAL DATASETN OP : CSV OF GIVEN NO OF ROWS

In [ ]:
import pandas as pd

# Define the path to the input CSV file
input_csv_path = "/content/nyayanumana_knowledge.csv"

# Define the path for the output CSV file
output_csv_path = "selected_datapoints.csv"

# Define the start and end row (inclusive) for extraction
start_row = 0 # Remember that pandas indexing starts from 0
end_row = 199  # To select the first 200 rows (0 to 199)

# Read the CSV file into a pandas DataFrame
try:
    # Use engine='python' and on_bad_lines='skip' to handle potential parsing errors
    df = pd.read_csv(input_csv_path, engine='python', on_bad_lines='skip')
    print(f"Successfully read {input_csv_path}")
except FileNotFoundError:
    print(f"Error: The file {input_csv_path} was not found.")
    df = None
except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")
    df = None


# Check if the DataFrame was loaded and has the specified rows
if df is not None:
    # Ensure end_row is within the DataFrame bounds
    if end_row >= len(df):
        print(f"Warning: end_row ({end_row}) is beyond the number of rows in the DataFrame ({len(df)}). Extracting up to the last row.")
        end_row = len(df) - 1
    if start_row < 0:
        print(f"Warning: start_row ({start_row}) is less than 0. Starting extraction from row 0.")
        start_row = 0
    if start_row > end_row:
        print(f"Error: start_row ({start_row}) is greater than end_row ({end_row}). No rows will be extracted.")
        selected_df = pd.DataFrame() # Create an empty DataFrame
    else:
        # Select the specified range of rows
        selected_df = df.iloc[start_row : end_row + 1] # Add 1 to end_row to make it inclusive
        print(f"Selected rows from {start_row} to {end_row}.")


    # Save the selected data points to a new CSV file
    if not selected_df.empty:
        try:
            # Set index=False to avoid writing the DataFrame index as a column in the CSV
            selected_df.to_csv(output_csv_path, index=False)
            print(f"Successfully saved the selected data points to {output_csv_path}")
        except Exception as e:
            print(f"Error saving the DataFrame to CSV: {e}")
    else:
        print("No data points selected to save.")

API KEYS

In [ ]:
import os

# Replace with your actual API keys
os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "763be8ff039562b47f718f61a7e7c05eb36e834004a995dcd964dd219b2cc37b"

print("Environment variables GROQ_API_KEY and SERPAPI_API_KEY have been set.")

INSTALLATIONS

In [ ]:
!pip install SerpApi
!pip install groq
!pip install google-search-results
!pip install google_search_results

EXTRACTING 'text' FIELD FROM THE CSV OP : EXTRACTED TEXT.JSON FILE CONTATINING TEXT COLUMNS OF THE CSV

In [ ]:
import csv
import ast
import json

def extract_text_from_csv(input_csv_path, output_json_path):
    """
    Extracts the 'text' field from each row of a CSV file and saves it
    as a list of dictionaries in a JSON file.

    The script assumes the text data is in the first column of the CSV,
    formatted as a string representation of a Python dictionary.

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_json_path (str): The path for the output JSON file.
    """
    extracted_data = []
    try:
        with open(input_csv_path, 'r', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)

            # Skip the header row
            next(csv_reader, None)

            for row in csv_reader:
                # Ensure the row is not empty
                if not row:
                    continue

                # The dictionary-like string is in the first column
                data_string = row[0]

                try:
                    # Safely evaluate the string to a Python dictionary
                    data_dict = ast.literal_eval(data_string)

                    # Extract the value from the 'text' key and append
                    if isinstance(data_dict, dict) and 'text' in data_dict:
                        extracted_data.append({'text': data_dict['text']})
                except (ValueError, SyntaxError) as e:
                    print(f"Could not parse row: {data_string[:100]}... Error: {e}")

        # Write the list of dictionaries to the output JSON file
        with open(output_json_path, 'w', encoding='utf-8') as jsonfile:
            json.dump(extracted_data, jsonfile, indent=4)

        print(f"Successfully extracted data to '{output_json_path}'")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == '__main__':
    # Define the input and output file names
    input_file = '/content/top_300_datapoints.csv'
    output_file = 'extracted_data.json'

    # Run the extraction function
    extract_text_from_csv(input_file, output_file)

CODE FOR BUILDING APPLICATION CORPUS
OP : APPLICATION CORPUS.CSV

In [ ]:
import csv
import os
import sys
import time
import requests
from bs4 import BeautifulSoup
from groq import Groq
from serpapi import GoogleSearch
import os
import json # Import the json library

# Replace with your actual API keys
os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "763be8ff039562b47f718f61a7e7c05eb36e834004a995dcd964dd219b2cc37b"

print("Environment variables GROQ_API_KEY and SERPAPI_API_KEY have been set.")

# ---------------- Configuration ----------------
GROQ_API_KEY ="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"
SERPAPI_KEY = "763be8ff039562b47f718f61a7e7c05eb36e834004a995dcd964dd219b2cc37b"

OUTPUT_CSV_FILE = "application_corpus.csv"

# Allowed legal source domains for filtering
LEGAL_DOMAINS = [
    "indiankanoon.org",
    "barandbench.com",
    "livelaw.in",
    "scobserver.in",
    "legalserviceindia.com",
]

# ---------------- Helper: Check API Keys ----------------
def check_api_keys():
    if not GROQ_API_KEY:
        print("ERROR: GROQ_API_KEY not set.")
        sys.exit(1)
    if not SERPAPI_KEY:
        print("ERROR: SERPAPI_KEY not set.")
        sys.exit(1)
    print("API keys verified.\n")


# ---------------- LLM Call Wrapper ----------------
def llm_call(prompt, task_description):
    try:
        print(f"\n[LLM] {task_description} ...")
        client = Groq(api_key=GROQ_API_KEY)
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="openai/gpt-oss-20b",
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Groq API error during '{task_description}': {e}")
        return ""


# ---------------- 1️⃣ Generate Search Query ----------------
def generate_search_query(original_text):
    prompt = f"""
You are generating a legal search query based on this legal context.

Context:
{original_text}

Generate a short, specific Google search query that would return Indian court judgments or legal discussions related to this context.
"""
    return llm_call(prompt, "Generate legal search query")


# ---------------- 2️⃣ Get Top Search Links (Modified) ----------------
def get_top_search_links(query, top_k=3):
    print(f"\n🔍 Searching for: {query}")
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": top_k * 3,  # retrieve extra to filter later
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    all_results = results.get("organic_results", [])
    if not all_results:
        print("No search results found.")
        return []

    filtered_links = []
    for result in all_results:
        link = result.get("link", "")
        if any(domain in link for domain in LEGAL_DOMAINS):
            filtered_links.append(link)
        if len(filtered_links) >= top_k:
            break

    print(f"✅ Retrieved {len(filtered_links)} valid legal links.")
    return filtered_links


# ---------------- 3️⃣ Scrape and Clean Page ----------------
def get_and_clean_page_content(url):
    try:
        print(f"Scraping: {url}")
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # Extract main content
        text = " ".join(p.get_text() for p in soup.find_all("p"))
        cleaned_text = " ".join(text.split())
        return cleaned_text[:4000]  # keep it concise
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""


# ---------------- 4️⃣ Summarize Content for Application ----------------
def summarize_content_for_application(retrieved_text, original_text):
    prompt = f"""
You are creating a legal 'application corpus' example.

Base legal principle:
{original_text}

Retrieved real-world legal case content:
{retrieved_text}

Summarize how the retrieved case applies or exemplifies the base principle.
Keep it concise (max 6 sentences).
"""
    return llm_call(prompt, "Summarize retrieved legal content")


# ---------------- 5️⃣ Generate Final Reasoning ----------------
def generate_final_reasoning(original_text, summary):
    prompt = f"""
You are an AI trained in Indian legal reasoning.

Base Principle:
{original_text}

Application Summary:
{summary}

Generate a brief reasoning statement connecting the base principle to the summarized application.
Focus on cause-effect legal reasoning.
"""
    return llm_call(prompt, "Generate final reasoning")


# ---------------- 6️⃣ Main Pipeline ----------------
def build_application_corpus(dataset):
    with open(OUTPUT_CSV_FILE, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Base Text", "Sources", "Retrieved Summary", "Final Reasoning"])

        for i, data in enumerate(dataset, start=1):
            print(f"\n============================")
            print(f"Processing #{i}")
            print("============================")

            original_text = data["text"]

            # Step 1: Generate search query
            query = generate_search_query(original_text)

            # Step 2: Retrieve top links
            links = get_top_search_links(query, top_k=3)
            if not links:
                writer.writerow([original_text, "", "", ""])
                continue

            # Step 3: Combine multiple page contents
            combined_text = ""
            for link in links:
                page_text = get_and_clean_page_content(link)
                if page_text:
                    combined_text += f"\n\nSource: {link}\n{page_text}"

            # Step 4: Summarize
            summary = summarize_content_for_application(combined_text, original_text)

            # Step 5: Generate reasoning
            reasoning = generate_final_reasoning(original_text, summary)

            # Step 6: Save in CSV
            writer.writerow([original_text, "; ".join(links), summary, reasoning])
            print("✅ Entry added to corpus.\n")

            # Delay between entries to avoid SerpApi rate limit
            time.sleep(3)


# ---------------- Execute ----------------
if __name__ == "__main__":
    check_api_keys()
    # Load data from the JSON file
    json_file_path = "/content/extracted_data.json"
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            nyayanumana_dataset = json.load(f)
        print(f"Successfully loaded data from {json_file_path}")
    except FileNotFoundError:
        print(f"Error: The file {json_file_path} was not found.")
        nyayanumana_dataset = []
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}. Please ensure the file is valid JSON.")
        nyayanumana_dataset = []


    build_application_corpus(nyayanumana_dataset)
    print(f"\n✅ Application corpus created: {OUTPUT_CSV_FILE}")